# Enough chit chat, let's make some models!

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import statsmodels.api as sm

In [3]:
from terra_mystica_models.features.model_subset import get_model_games_df

In [4]:
df = get_model_games_df()

In [5]:
len(df)

49740

In [6]:
df.columns

Index(['date', 'file', 'score_turn_1', 'score_turn_2', 'score_turn_3',
       'score_turn_4', 'score_turn_5', 'score_turn_6', 'BON1', 'BON2', 'BON3',
       'BON4', 'BON5', 'BON6', 'BON7', 'BON8', 'BON9', 'BON10',
       'player_1_faction', 'player_2_faction', 'player_3_faction',
       'player_4_faction', 'player_1_user', 'player_2_user', 'player_3_user',
       'player_4_user', 'player_1_vp', 'player_2_vp', 'player_3_vp',
       'player_4_vp', 'player_1_vp_margin', 'player_2_vp_margin',
       'player_3_vp_margin', 'player_4_vp_margin'],
      dtype='object')

OK, we have 34,000 ish games. That should give us plenty of room to add parameters to a model, but we should start simple.

My first idea is to try and fit victory point margin at the player/game level. So I can turn each game into four observations. This will require a bit more transformation, but let's see what it gives

In [7]:
score_cols = [f"score_turn_{i}" for i in range(1, 7)]
bon_cols = [f"BON{i}" for i in range(1, 11)]

game_level_info = df.reindex(columns=score_cols + bon_cols).reset_index().rename(columns={"index": "game_name"})

In [8]:
game_level_info.head()

,game_name,score_turn_1,score_turn_2,score_turn_3,score_turn_4,score_turn_5,score_turn_6,BON1,BON2,BON3,BON4,BON5,BON6,BON7,BON8,BON9,BON10
0,10,4 AIR -> 1 SPADE | TP >> 3,1 EARTH -> 1 C | SPADE >> 2,2 AIR -> 1 W | SA/SH >> 5,4 WATER -> 1 SPADE | TP >> 3,4 FIRE -> 4 PW | D >> 2,2 FIRE -> 1 W | SA/SH >> 5,False,True,True,True,True,True,True,False,True,False
1,21,1 EARTH -> 1 C | SPADE >> 2,4 FIRE -> 4 PW | D >> 2,4 WATER -> 1 SPADE | TP >> 3,4 WATER -> 1 P | D >> 2,4 AIR -> 1 SPADE | TP >> 3,2 AIR -> 1 W | SA/SH >> 5,False,True,True,True,True,True,True,True,False,False
2,5,4 WATER -> 1 P | D >> 2,2 FIRE -> 1 W | SA/SH >> 5,4 EARTH -> 1 SPADE | TOWN >> 5,4 AIR -> 1 SPADE | TP >> 3,1 EARTH -> 1 C | SPADE >> 2,4 WATER -> 1 SPADE | TP >> 3,True,True,True,True,False,False,True,True,True,False
3,gareth4,2 FIRE -> 1 W | SA/SH >> 5,4 WATER -> 1 P | D >> 2,4 AIR -> 1 SPADE | TP >> 3,4 WATER -> 1 SPADE | TP >> 3,4 FIRE -> 4 PW | D >> 2,2 AIR -> 1 W | SA/SH >> 5,False,True,False,True,True,True,True,True,True,False
4,skelly1,4 WATER -> 1 SPADE | TP >> 3,4 WATER -> 1 P | D >> 2,2 AIR -> 1 W | SA/SH >> 5,1 EARTH -> 1 C | SPADE >> 2,4 FIRE -> 4 PW | D >> 2,4 AIR -> 1 SPADE | TP >> 3,True,True,True,True,True,True,False,True,False,False


In [9]:
def player_n_frame(base_df, n):
    player_dict = {
        f"player_{n}_faction": "faction",
        f"player_{n}_vp_margin": "vp_margin"
    }
    player_n_df = (
        base_df
        .rename(columns=player_dict)
        .reindex(columns=["faction", "vp_margin"])
        .assign(player_num=n)
        .reset_index()
        .rename(columns={"index": 'game_name'})
    )
    return player_n_df
player_df = pd.concat([player_n_frame(df, i) for i in range(1, 5)])

In [10]:
len(player_df) == len(df) * 4

True

In [11]:
player_df.head()

,game_name,faction,vp_margin,player_num
0,10,witches,-23.00,1
1,21,nomads,5.75,1
2,5,nomads,-9.75,1
3,gareth4,mermaids,8.00,1
4,skelly1,chaosmagicians,-13.50,1


In [12]:
recombined_df = player_df.merge(game_level_info, on="game_name")

In [13]:
len(recombined_df) == len(player_df)

True

In [14]:
recombined_df.head()

,game_name,faction,vp_margin,player_num,score_turn_1,score_turn_2,score_turn_3,score_turn_4,score_turn_5,score_turn_6,BON1,BON2,BON3,BON4,BON5,BON6,BON7,BON8,BON9,BON10
0,10,witches,-23.00,1,4 AIR -> 1 SPADE | TP >> 3,1 EARTH -> 1 C | SPADE >> 2,2 AIR -> 1 W | SA/SH >> 5,4 WATER -> 1 SPADE | TP >> 3,4 FIRE -> 4 PW | D >> 2,2 FIRE -> 1 W | SA/SH >> 5,False,True,True,True,True,True,True,False,True,False
1,10,chaosmagicians,-7.00,2,4 AIR -> 1 SPADE | TP >> 3,1 EARTH -> 1 C | SPADE >> 2,2 AIR -> 1 W | SA/SH >> 5,4 WATER -> 1 SPADE | TP >> 3,4 FIRE -> 4 PW | D >> 2,2 FIRE -> 1 W | SA/SH >> 5,False,True,True,True,True,True,True,False,True,False
2,10,swarmlings,-16.00,3,4 AIR -> 1 SPADE | TP >> 3,1 EARTH -> 1 C | SPADE >> 2,2 AIR -> 1 W | SA/SH >> 5,4 WATER -> 1 SPADE | TP >> 3,4 FIRE -> 4 PW | D >> 2,2 FIRE -> 1 W | SA/SH >> 5,False,True,True,True,True,True,True,False,True,False
3,10,darklings,46.00,4,4 AIR -> 1 SPADE | TP >> 3,1 EARTH -> 1 C | SPADE >> 2,2 AIR -> 1 W | SA/SH >> 5,4 WATER -> 1 SPADE | TP >> 3,4 FIRE -> 4 PW | D >> 2,2 FIRE -> 1 W | SA/SH >> 5,False,True,True,True,True,True,True,False,True,False
4,21,nomads,5.75,1,1 EARTH -> 1 C | SPADE >> 2,4 FIRE -> 4 PW | D >> 2,4 WATER -> 1 SPADE | TP >> 3,4 WATER -> 1 P | D >> 2,4 AIR -> 1 SPADE | TP >> 3,2 AIR -> 1 W | SA/SH >> 5,False,True,True,True,True,True,True,True,False,False


Ok, so now we have a series of observations that tell us what faction a player chose, their number, the scoring and bonus tiles available, and how far above or below average they scored. We'll have to do some transformations on the factions and scoring tiles before we can use them in a model. I'm kind of regretting giving the scoring tiles such long names. I'm going to switch them back to just numbers for now too

In [15]:
# This was the original transformation, we'll just reverse it
score_name_dict = {
    "SCORE1": "1 EARTH -> 1 C | SPADE >> 2",
    "SCORE2": "4 EARTH -> 1 SPADE | TOWN >> 5",
    "SCORE3": "4 WATER -> 1 P | D >> 2",
    "SCORE4": "2 FIRE -> 1 W | SA/SH >> 5",
    "SCORE5": "4 FIRE -> 4 PW | D >> 2",
    "SCORE6": "4 WATER -> 1 SPADE | TP >> 3",
    "SCORE7": "2 AIR -> 1 W | SA/SH >> 5",
    "SCORE8": "4 AIR -> 1 SPADE | TP >> 3",
    "SCORE9": "1 CULT_P -> 2 C | TE >> 4",
}
easy_score_names = {value: key for key, value in score_name_dict.items()}

In [16]:
easy_score_names

{'1 EARTH -> 1 C | SPADE >> 2': 'SCORE1',
 '4 EARTH -> 1 SPADE | TOWN >> 5': 'SCORE2',
 '4 WATER -> 1 P | D >> 2': 'SCORE3',
 '2 FIRE -> 1 W | SA/SH >> 5': 'SCORE4',
 '4 FIRE -> 4 PW | D >> 2': 'SCORE5',
 '4 WATER -> 1 SPADE | TP >> 3': 'SCORE6',
 '2 AIR -> 1 W | SA/SH >> 5': 'SCORE7',
 '4 AIR -> 1 SPADE | TP >> 3': 'SCORE8',
 '1 CULT_P -> 2 C | TE >> 4': 'SCORE9'}

In [17]:
for i in range(1, 7):
    recombined_df[f"score_turn_{i}"] = recombined_df[f"score_turn_{i}"].map(easy_score_names)

In [18]:
recombined_df.head()

,game_name,faction,vp_margin,player_num,score_turn_1,score_turn_2,score_turn_3,score_turn_4,score_turn_5,score_turn_6,BON1,BON2,BON3,BON4,BON5,BON6,BON7,BON8,BON9,BON10
0,10,witches,-23.00,1,SCORE8,SCORE1,SCORE7,SCORE6,SCORE5,SCORE4,False,True,True,True,True,True,True,False,True,False
1,10,chaosmagicians,-7.00,2,SCORE8,SCORE1,SCORE7,SCORE6,SCORE5,SCORE4,False,True,True,True,True,True,True,False,True,False
2,10,swarmlings,-16.00,3,SCORE8,SCORE1,SCORE7,SCORE6,SCORE5,SCORE4,False,True,True,True,True,True,True,False,True,False
3,10,darklings,46.00,4,SCORE8,SCORE1,SCORE7,SCORE6,SCORE5,SCORE4,False,True,True,True,True,True,True,False,True,False
4,21,nomads,5.75,1,SCORE1,SCORE5,SCORE6,SCORE3,SCORE8,SCORE7,False,True,True,True,True,True,True,True,False,False


OK, that will make the columns easier to read once we start making dummy variables. 

I'd also like to be able to consider the option that having a given scoring tile matters, but its order doesn't. At least at first. That will certainly be easier to check for to start. Let's add in some columns that will allow that

In [19]:
score_cols = [f"score_turn_{i}" for i in range(1, 7)]
possible_scores = list(score_name_dict.keys())
for score in possible_scores:
    recombined_df[score] = recombined_df[score_cols].isin([score]).any(axis="columns")

In [20]:
drop_cols = ["faction", "game_name"]
drop_cols.extend([f"score_turn_{i}" for i in range(1, 7)])
dummy_cols = ["player_num", "faction"]
dummy_cols.extend([f"score_turn_{i}" for i in range(1, 7)])
dummy_frames = [pd.get_dummies(recombined_df[col], prefix=col, drop_first=True) for col in dummy_cols]
predict_list = dummy_frames + [recombined_df.drop(columns=drop_cols)]
predict_df = pd.concat(predict_list, axis="columns")

In [21]:
len(predict_df) == len(recombined_df)

True

In [22]:
predict_df.columns

Index(['player_num_2', 'player_num_3', 'player_num_4', 'faction_auren',
       'faction_chaosmagicians', 'faction_cultists', 'faction_darklings',
       'faction_dwarves', 'faction_engineers', 'faction_fakirs',
       'faction_giants', 'faction_halflings', 'faction_mermaids',
       'faction_nomads', 'faction_swarmlings', 'faction_witches',
       'score_turn_1_SCORE2', 'score_turn_1_SCORE3', 'score_turn_1_SCORE4',
       'score_turn_1_SCORE5', 'score_turn_1_SCORE6', 'score_turn_1_SCORE7',
       'score_turn_1_SCORE8', 'score_turn_1_SCORE9', 'score_turn_2_SCORE2',
       'score_turn_2_SCORE3', 'score_turn_2_SCORE4', 'score_turn_2_SCORE5',
       'score_turn_2_SCORE6', 'score_turn_2_SCORE7', 'score_turn_2_SCORE8',
       'score_turn_2_SCORE9', 'score_turn_3_SCORE2', 'score_turn_3_SCORE3',
       'score_turn_3_SCORE4', 'score_turn_3_SCORE5', 'score_turn_3_SCORE6',
       'score_turn_3_SCORE7', 'score_turn_3_SCORE8', 'score_turn_3_SCORE9',
       'score_turn_4_SCORE2', 'score_turn_4_SCORE

Ok, on top of that I'm going to want some interaction terms, because what we really want to know is how being a particular faction modifies the value of certain scoring tiles or bonus tiles.

In [23]:
faction_cols = [col for col in predict_df.columns if col.startswith("faction_")]
non_interact_cols = ["vp_margin", "player_num"]
non_interact_cols.extend(faction_cols)
interact_cols = [col for col in predict_df.columns if col not in non_interact_cols]

Ok, that many interactions is going to get huge, so actually let's do some simpler modelling to start. Just to get a flavour of things

In [24]:
y = predict_df["vp_margin"]
x_cols = ["player_num"] + faction_cols
X = sm.add_constant(predict_df[x_cols])

In [25]:
simple_model = sm.OLS(y, X).fit()
simple_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              vp_margin   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     464.2
Date:                Sat, 28 Mar 2020   Prob (F-statistic):               0.00
Time:                        16:10:28   Log-Likelihood:            -8.4639e+05
No. Observations:              198960   AIC:                         1.693e+06
Df Residuals:                  198945   BIC:                         1.693e+06
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
const                     -3.7683      0.220    -17.166      0.000      -4.199      -3.338
player_num                -0.3301      0.035     -9.372      0.000      -0.399      -0.261
faction_auren             -0.2274      0.315     -0.723      0.470      -0.844       0.389
faction_chaosmagicians     4.0712      0.233     17.444      0.000       3.614       4.529
faction_cultists           6.7553      0.257     26.303      0.000       6.252       7.259
faction_darklings          9.7877      0.225     43.527      0.000       9.347      10.228
faction_dwarves            3.3857      0.284     11.904      0.000       2.828       3.943
faction_engineers          6.3924      0.240     26.622      0.000       5.922       6.863
faction_fakirs            -4.2379      0.396    -10.692      0.000      -5.015      -3.461
faction_giants            -1.0285      0.298     -3.446      0.001      -1.613      -0.443
faction_halflings          2.8698      0.251     11.445      0.000       2.378       3.361
faction_mermaids           4.2740      0.239     17.872      0.000       3.805       4.743
faction_nomads             3.1887      0.234     13.650      0.000       2.731       3.647
faction_swarmlings         2.6398      0.249     10.585      0.000       2.151       3.129
faction_witches            4.4320      0.240     18.481      0.000       3.962       4.902
==============================================================================
Omnibus:                     1026.977   Durbin-Watson:                   2.495
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1251.219
Skew:                          -0.107   Prob(JB):                    2.00e-272
Kurtosis:                       3.325   Cond. No.                         59.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Ok, quick interpretation. Looks like going later in the game gives you a slight disadvantage (the -0.33 coefficient). There is definitely variability within the factions. Some of this might be related to stronger players choosing particular factions, but it looks like on average if you see a random game you should bet on the darklings. Worst faction appears to be the fakirs.

OK. I'm going to push this up and get some feedback. Assuming these results look good I'll refactor some of the code into modules and start trying to build a model to answer the actual question (what faction should you pick given a set of bonus and score cards?)